# Basic collaborative filtering algorithm for movielens 100k data
- reference: https://acodeforthought.wordpress.com/2016/12/29/building-a-recommender-system-on-user-user-collaborative-filtering-movielens-dataset/

In [1]:
import pandas as pd
import numpy as np
import math

## Data import
- movielens 100k dataset 으로부터 3가지 데이터 셋을 불러온다
    - **u.user**: 각 user에 대한 정보(demographic information about each user)
    - **u.item**: 각 영화에 대한 정보(description regarding each item, i.e., each movie)
    - **u.data**: 각 user의 영화에 대한 평가(rating of each user regarding an item)

In [2]:
# 각 데이터(user, item, rating)의 열 이름(column name)을 정의한다
user_cols = ['user id','age','gender','occupation','zip code']
item_cols = ['movie id','movie title','release date','video release date','IMDb URL','unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance ','Sci-Fi','Thriller','War' ,'Western']
rating_cols = ['user id','movie id','rating','timestamp']

In [3]:
# pandas의 read_csv() 함수를 활용해 각 데이터셋을 불러온다
users = pd.read_csv('ml-100k/u.user', sep = '|', names = user_cols, encoding = 'latin-1')       
items = pd.read_csv('ml-100k/u.item', sep = '|', names = item_cols, encoding = 'latin-1')
ratings = pd.read_csv('ml-100k/u.data', sep = '\t', names = rating_cols, encoding = 'latin-1')

### Dataset description

In [4]:
print(users.shape)     # 943 users in total
print(items.shape)     # 1682 items (i.e., movies) in total
print(ratings.shape)   # 100000 ratings in total

(943, 5)
(1682, 24)
(100000, 4)


In [5]:
print(users.head())    # users dataframe의 첫 5 행의 데이터

   user id  age gender  occupation zip code
0        1   24      M  technician    85711
1        2   53      F       other    94043
2        3   23      M      writer    32067
3        4   24      M  technician    43537
4        5   33      F       other    15213


In [6]:
print(items.head())    # items dataframe의 첫 5 행의 데이타

   movie id        movie title release date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Childrens   ...     Fantasy  Film-Noir  Horror  \
0          0          1          1   ...           0          0       0   
1       

In [7]:
print(ratings.head())    # ratings dataframe의 첫 5 행의 데이타

   user id  movie id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596


## Data preprocessing

### train-test split
- 학습 데이터와 검증 데이터로 전체 데이터셋을 나눈다
- user id를 기준으로 ratings 데이터를 sorting한 후 99832번째 관측치까지는 학습 데이터로, 99833번째 관측치부터는 검증 데이터로 할당한다
- 그 결과 user id 1부터 942까지의 관측치는 학습 데이터가 되고, user id 943과 관련된 관측치는 검증 데이터가 된다
- 즉, 우리는 1~942 까지의 user의 데이터를 통해 943번째 user의 선호를 예측해 본다

In [8]:
ratings_train = (ratings.sort_values('user id'))[:99832]    
ratings_test = (ratings.sort_values('user id'))[99833:]

In [9]:
print(ratings_train.shape)
print(ratings_test.shape)

(99800, 4)
(200, 4)


In [10]:
print(ratings_train.head())
print()
print(ratings_train.tail())
print()
print(ratings_test.head())
print()
print(ratings_test.tail())

       user id  movie id  rating  timestamp
66567        1        55       5  875072688
62820        1       203       4  878542231
10207        1       183       5  875072262
9971         1       150       5  876892196
22496        1        68       4  875072688
       user id  movie id  rating  timestamp
96445      942       678       3  891282673
88898      942       662       4  891283517
73787      942       172       5  891282963
87107      942       879       4  891282539
73374      942       969       4  891282817


In [ ]:
# pandas의 as_matrix() 함수로 dataframe을 numpy array로 바꾼다
ratings_train = ratings_train.as_matrix(columns = ['user id', 'movie id', 'rating'])
ratings_test = ratings_test.as_matrix(columns = ['user id', 'movie id', 'rating'])

In [12]:
# numpy array로 바꾸어도 shape는 그대로 유지된다
print(ratings_train.shape)
print(ratings_test.shape)

(99800, 3)
(200, 3)


### combining user-rating data
- 각 user의 rating 정보를 담은 리스트를 생성한다
- 결과물인 users_list 리스트는 1부터 942까지의 각 user가 어떠한 영화에 대해 rating을 어떻게 했는지와 관한 정보를 담고 있다

In [13]:
users_list = []
for i in range(1, users.shape[0]):
    temp = []
    for j in range(0, len(ratings_train)):
        if ratings_train[j][0] == i:
            temp.append(ratings_train[j])
        else:
            break
    ratings_train = ratings_train[j:]
    users_list.append(temp)

print(len(users_list))

943


In [14]:
print(users_list)

[[], [array([ 1, 55,  5], dtype=int64), array([  1, 203,   4], dtype=int64), array([  1, 183,   5], dtype=int64), array([  1, 150,   5], dtype=int64), array([ 1, 68,  4], dtype=int64), array([  1, 201,   3], dtype=int64), array([  1, 157,   4], dtype=int64), array([  1, 184,   4], dtype=int64), array([  1, 210,   4], dtype=int64), array([  1, 163,   4], dtype=int64), array([  1, 271,   2], dtype=int64), array([  1, 146,   4], dtype=int64), array([  1, 176,   5], dtype=int64), array([  1, 144,   4], dtype=int64), array([ 1, 53,  3], dtype=int64), array([  1, 160,   4], dtype=int64), array([ 1, 33,  4], dtype=int64), array([ 1, 44,  5], dtype=int64), array([ 1, 97,  3], dtype=int64), array([ 1, 14,  5], dtype=int64), array([  1, 263,   1], dtype=int64), array([1, 4, 3], dtype=int64), array([ 1, 17,  3], dtype=int64), array([  1, 265,   4], dtype=int64), array([ 1, 25,  4], dtype=int64), array([ 1, 37,  2], dtype=int64), array([  1, 251,   4], dtype=int64), array([  1, 195,   5], dtype=in

## Similarity calculation
- train user (1-942)와 test user (943) 간의 유사도(거리)를 계산한다
- user 943과 높은 유사도(낮은 euclidean score)를 가진 user들을 추려낸다
- 추려낸 user들이 높게 rating을 매긴 영화(item)들을 추천한다

### EuclideanScore() function
- 두 user 간의 유사도(거리)를 계산할 수 있는 EuclideanScore() 함수를 정의한다
- 두 user 간에 4회 이상 동일한 영화를 rating한 경우에만 거리를 계산한다
- 두 user 간에 4회 이하로 동일한 영화를 rating한 경우에는 거리를 1,000 (임의의 큰 정수)로 정의한다

In [15]:
def EucledianScore(train_user, test_user):
    s = 0
    count = 0
    for i in test_user:
        score = 0
        for j in train_user:
            if(int(i[1]) == int(j[1])):
                score= ((float(i[2])-float(j[2]))*(float(i[2])-float(j[2])))
                count= count + 1        
            s = s + score
    if(count<4):
        s = 1000000           
    return(math.sqrt(s))

### calculate Euclidean Scores
- user 943과 user 1~942 간의 Euclidean Score를 계산하여 낮은 순서대로 정렬한다

In [16]:
score_list = []
for i in range(942):
    score_list.append([i+1, EucledianScore(users_list[i], ratings_test)])   

In [18]:
score = pd.DataFrame(score_list, columns = ['user id', 'Euclidean Score'])
score = score.sort_values(by = 'Euclidean Score')
score.reset_index()
print(score.shape)

(943, 2)


In [19]:
print(score.head())    # 유사도가 가장 높은 5명을 출력
print()
print(score.tail())    # 유사도가 가장 낮은 5명을 출력

     user id  Euclidean Score
310      311         1.732051
139      140         3.872983
46        47         4.000000
209      210         4.242641
558      559         4.582576
725      726         4.690416
100      101         5.000000
242      243         5.000000
414      415         5.099020
112      113         5.291503
856      857         5.656854
754      755         5.744563
673      674         5.744563
4          5         5.830952
134      135         5.916080
797      798         6.244998
475      476         6.557439
616      617         6.557439
800      801         6.557439
464      465         6.633250
266      267         6.782330
636      637         6.928203
439      440         7.000000
304      305         7.141428
581      582         7.416198
516      517         7.483315
832      833         7.681146
677      678         7.681146
376      377         7.810250
520      521         7.874008
..       ...              ...
400      401      1000.000000
818      8

In [20]:
# pandas dataframe을 numpy array로 변환한다
score_matrix = score.as_matrix()

## Recommendation for user 943
- user 943을 위한 추천 영화 리스트를 작성한다
- user 943과 가장 유사한 user 310이 좋게 평가한 영화 중 user 943이 평가하지 않은 영화들을 추천한다

In [21]:
# user 310이 평가한 모든 영화 정보를 담기 위한 full_list와 
# user 943과 user 310이 공히 평가한 영화 정보를 담기 위한 common_list를 생성한다
user = int(score_matrix[0][0])    
common_list = []
full_list = []

In [24]:
# common_list와 full_list를 채워넣는다
for i in ratings_test:
    for j in users_list[user-1]:
        if int(i[1]) == int(j[1]):
            common_list.append(int(j[1]))
        full_list.append(j[1])

In [25]:
# 각 리스트를 집합으로 변환한다
common_list = set(common_list)
full_list = set(full_list)

In [ ]:
# 추천 영화를 추려내기 위해 user 310이 평가한 영화 중에
# user 943이 이미 본 영화들을 배제한다
recommendation = full_list.difference(common_list)    # recommendation = full_list - common_list

In [28]:
item_list = (((pd.merge(items, ratings).sort_values(by = 'movie id')).groupby('movie title')))['movie id', 'movie title', 'rating']
item_list = item_list.mean()
print(item_list)

In [ ]:
item_list['movie title'] = item_list.index
item_list = item_list.as_matrix()

In [29]:
recommendation_list = []
for i in recommendation:
    recommendation_list.append(item_list[i-1])

In [30]:
# 추천 영화를 평균 평점(mean rating)이 높은 순으로 출력한다
recommendation = (pd.DataFrame(recommendation_list, columns = ['movie id', 'mean rating', 'movie title'])).sort_values(by = 'mean rating', ascending = False)
print(recommendation[['mean rating', 'movie title']])    # 추천 영화의 평균 평점과 제목을 출력한다

    mean rating                                        movie title
6      4.292929                                Citizen Kane (1941)
2      4.125000                              A Chef in Love (1996)
15     4.000000                            Butcher Boy, The (1998)
12     3.930514          Indiana Jones and the Last Crusade (1989)
5      3.839050                                 Chasing Amy (1997)
10     3.792899                         In the Line of Fire (1993)
3      3.648352                                      Casino (1995)
16     3.600000                         Murder in the First (1995)
11     3.545455                                     Stalker (1979)
4      3.166667  Flower of My Secret, The (Flor de mi secreto, ...
13     3.105263                                    Bad Boys (1995)
9      2.802632                      Brady Bunch Movie, The (1995)
7      2.750000                           Ladybird Ladybird (1994)
14     2.720930                               Pete's Dragon (1